In [19]:
import random
import multiprocessing
import pandas as pd
from deap import base
from deap import creator
from deap import tools
from datetime import datetime, timedelta

In [20]:
# Load files
file_name='RECINTO.csv'
RECINTO = pd.read_csv('./Data/'+file_name)
file_name='EVENTO.csv'
EVENTO = pd.read_csv('./Data/'+file_name)
file_name='PERFILES.csv'
PERFILES = pd.read_csv('./Data/'+file_name)
file_name='PREFERENCIAS_US1.csv'
PREFERENCIAS = pd.read_csv('./Data/'+file_name)

In [21]:
# Size of the individual
SIZE_INDIVIDUAL = max(EVENTO['Id_Recurso'])

# Number of generations
NUM_GENERATIONS = 60

# Optimal value to obtain (not reached)
MAX_VALUE = SIZE_INDIVIDUAL + 10 * SIZE_INDIVIDUAL

# Poblation members
SIZE_POBLATION = 4000

# CXPB  is the probability with which two individuals are crossed
CXPB = 0.5

# MUTPB is the probability for mutating an individual
MUTPB = 0.2

In [22]:
# Define the fitness function for maximizing the result
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Creation of the individual with the fitness function
creator.create("Individual", list, fitness=creator.FitnessMax)

C:\ProgramData\Anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\ProgramData\Anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [23]:
toolbox = base.Toolbox()

# Attribute generator having only 0 or 1 in the body of the individual
toolbox.register("attr_bool", random.randint, 0, 0)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, SIZE_INDIVIDUAL)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [24]:
# Enable multiprocessing
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

In [25]:
# Fitness function
def evalOneMax(individual):
    summa = 0
    if not find_overlapping(individual):
        summa = sum(individual)
        for i in range(0, len(individual)):
            if individual[i] == 1:
                summa = summa + PREFERENCIAS[PREFERENCIAS['Id_Recurso'] == i].Preferencia.values[0]
    return summa,

In [26]:
# Overlapping function to detect time problems in resources
def find_overlapping(individual):
    found = False
    i = 0
    while i < len(individual) and not found:
        # If the resource is selected for the algorithm.
        if individual[i] == 1:
            j = 0
            while j < len(individual) and not found:
                if i != j and individual[j] == 1:
                    # Get date for the first resource.
                    date_1 = datetime.strptime(EVENTO[EVENTO['Id_Recurso'] == i].Fecha_1.values[0], '%m/%d/%Y')
                    
                    # Get date for the second resource.
                    date_2 = datetime.strptime(EVENTO[EVENTO['Id_Recurso'] == j].Fecha_1.values[0], '%m/%d/%Y')
                    
                    # Evaluate if the resource is in the same day.
                    if (date_1 - date_2).days == 0:
                        # Get open time for the first resource.
                        ha1 = datetime.strptime(EVENTO[EVENTO['Id_Recurso'] == i].Hora_apertura_1.values[0], '%H:%M').time()
                        
                        # Get close time for the first resource.
                        hc1 = datetime.strptime(EVENTO[EVENTO['Id_Recurso'] == i].Hora_cierre_1.values[0], '%H:%M').time()
                        
                        # Get open time for the second resource.
                        ha2 = datetime.strptime(EVENTO[EVENTO['Id_Recurso'] == j].Hora_apertura_1.values[0], '%H:%M').time()
                        
                        # Get close time for the second resource.
                        hc2 = datetime.strptime(EVENTO[EVENTO['Id_Recurso'] == j].Hora_cierre_1.values[0], '%H:%M').time()
                        diff_hour = (hc2.hour - ha2.hour)
                        if diff_hour < 6:
                            if (ha1 == ha2) or (ha2 < ha1 < hc2) or (ha1 < ha2 < hc1) or (ha2 < ha1 < hc1 < hc2) \
                            or (ha1 < ha2 < hc2 < hc1):
                                found = True
                j = j + 1
        i = i + 1

    return found

In [27]:
best = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
if find_overlapping(best):
    print("Overlapping")
else:
    print("No overlapping")
    
evalOneMax(best)

No overlapping


(183.0,)

In [28]:
# Record the allowed operations
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0.0))

In [29]:
# Initial population
pop = toolbox.population(n=SIZE_POBLATION)

In [30]:
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))

In [31]:
# Store the values of the fitness function for each individual
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [32]:
# Function for calculating the stats each generation
def calculate_statistics(fits, g=0):
    print("-- Generation %i --" % g)   
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)

In [33]:
# Gather all the fitnesses in one list and print the stats
fits = [ind.fitness.values[0] for ind in pop]
calculate_statistics(fits)

-- Generation 0 --
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0


In [34]:
# Variable keeping track of the number of generations
g = 0

# Begin the evolution
while max(fits) < MAX_VALUE and g < NUM_GENERATIONS:
    # A new generation
    g = g + 1
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    pop[:] = offspring
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    calculate_statistics(fits, g)

-- Generation 1 --
  Min 0.0
  Max 46.0
  Avg 0.2856666666666667
  Std 2.4587384073047613
-- Generation 2 --
  Min 0.0
  Max 55.0
  Avg 1.208
  Std 4.529025207849183
-- Generation 3 --
  Min 0.0
  Max 51.0
  Avg 2.740666666666667
  Std 6.216221753516271
-- Generation 4 --
  Min 0.0
  Max 51.0
  Avg 5.310666666666667
  Std 8.158930866779597
-- Generation 5 --
  Min 0.0
  Max 56.0
  Avg 8.339
  Std 9.786389136618947
-- Generation 6 --
  Min 0.0
  Max 62.0
  Avg 11.361333333333333
  Std 11.04107957684493
-- Generation 7 --
  Min 0.0
  Max 56.0
  Avg 13.770666666666667
  Std 12.406479740665986
-- Generation 8 --
  Min 0.0
  Max 56.0
  Avg 16.522
  Std 14.100006477539885
-- Generation 9 --
  Min 0.0
  Max 56.0
  Avg 19.097666666666665
  Std 15.691721635591454
-- Generation 10 --
  Min 0.0
  Max 62.0
  Avg 20.315666666666665
  Std 17.316794003381677
-- Generation 11 --
  Min 0.0
  Max 66.0
  Avg 22.822666666666667
  Std 18.82512203295255
-- Generation 12 --
  Min 0.0
  Max 66.0
  Avg 26.8206

-- Generation 98 --
  Min 0.0
  Max 81.0
  Avg 64.29033333333334
  Std 32.755804165911655
-- Generation 99 --
  Min 0.0
  Max 81.0
  Avg 64.91766666666666
  Std 32.29485337566276
-- Generation 100 --
  Min 0.0
  Max 81.0
  Avg 63.41166666666667
  Std 33.384769939133754


In [37]:
# Find and print best individual:
best_index = fits.index(max(fits))
best_individual = pop[best_index]
print(len(best_individual))
print("Best Individual = ", best_individual, "\n")

109
Best Individual =  [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 



In [36]:
for i in range(0, len(best_individual)):
    if best_individual[i] == 1:
        print(i, EVENTO[EVENTO['Id_Recurso'] == i].Fecha_1.values[0], EVENTO[EVENTO['Id_Recurso'] == i].Hora_apertura_1.values[0], EVENTO[EVENTO['Id_Recurso'] == i].Hora_cierre_1.values[0], EVENTO[EVENTO['Id_Recurso'] == i].Nombre.values[0])

2 10/02/2023 11:00 12:00 Firma de discos de Sabina
8 10/02/2023 16:00 16:30 Firma de discos de Bombay
17 10/02/2023 19:00 20:00 Concierto de Arnau Griso
31 10/02/2023 17:00 18:00 Concierto Maluma
35 10/02/2023 10:00 10:30 Firma Kase.O
46 10/02/2023 18:00 18:30 Firma Sevendust
54 10/02/2023 20:00 21:00 Concierto sinfonico Real Conservatorio Superior de Música de Madrid
55 10/02/2023 21:00 22:00 Concierto coro Escuela Superior de Música Reina Sofía
91 10/02/2023 12:00 16:00 Taco Bell
